In [1]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/intentconan-test-csv/test.csv
/kaggle/input/intentconan/flan_t5_counterspeech.csv


In [4]:
import pandas as pd
import torch
from tqdm import tqdm  # For progress tracking
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM


file_path = "/kaggle/input/intentconan-test-csv/test.csv"
df = pd.read_csv(file_path)
df.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,index,hatespeech,csType,counterspeech,Suggest,Relevance,Aggressive,Complexity,Comments,source,...,hatespeechTarget,powerDynamics,prompt_offensiveness,prompt_target_group,prompt_speaker_intent,prompt_power_dynamics,prompt_implication,prompt_emotional_reaction,prompt_cognitive_reaction,prompt_cs_generation
0,20,"Women are basically childlike, they remain thi...",Informative,The characteristics you are referring to are a...,3.0,4.0,2.0,3.0,NaN,Human,...,women,gender,Analyze the offensiveness of the statement: Wo...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
1,22,"Women are basically childlike, they remain thi...",Denouncing,Nobody apart from people who spread misinforma...,2.0,3.0,3.0,2.0,NaN,Human,...,women,gender,Analyze the offensiveness of the statement: Wo...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
2,23,"Women are basically childlike, they remain thi...",Positive,It is a question of survival and happiness to ...,2.0,2.0,2.0,2.0,NaN,Human,...,women,gender,Analyze the offensiveness of the statement: Wo...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
3,48,Minorities vote to strip wealth and power from...,Informative,This is incorrect. jews could never be able to...,3.0,4.0,3.0,3.0,NaN,Human,...,jews,religion,Analyze the offensiveness of the statement: Mi...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
4,49,Minorities vote to strip wealth and power from...,Questioning,Where is your evidence?,1.0,1.0,3.0,1.0,NaN,Human,...,jews,religion,Analyze the offensiveness of the statement: Mi...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print( "Using device:", device)


Using device: cuda


In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd
from tqdm import tqdm  # Progress bar

model_name = "google/flan-t5-large"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device) 

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
print("Model Device:", next(model.parameters()).device)  # Should print "cuda"


Model Device: cuda:0


In [7]:
def generate_counterspeech_flan_t5(hate_speech, intent):
    """Generates counterspeech using Flan-T5 on GPU."""
    prompt = f"""
        You are an expert in counterspeech generation. Your task is to generate an effective {intent} counterspeech response to the following hate speech.

        ### Hate Speech:
        {hate_speech}

        ### Expected Response:
        Your response should be a strong {intent} counterspeech that directly addresses the hate speech.
    """

    inputs = tokenizer(prompt, return_tensors="pt").to(device)  # Move inputs to GPU
    with torch.no_grad():  # Disable gradient tracking for faster inference
        outputs = model.generate(
            **inputs, 
            max_new_tokens=250,  # Prevent cutoff
            repetition_penalty=1.2, 
            no_repeat_ngram_size=3,  
            temperature=0.7,  
            top_k=50,  
            top_p=0.9
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

df["FlanT5_counterspeech"] = ""

print("\n Generating counterspeech using Flan-T5 on GPU...")


for i in tqdm(range(len(df)), desc="Processing Hate Speech with Flan-T5"):
    df.at[i, "FlanT5_counterspeech"] = generate_counterspeech_flan_t5(
        df.at[i, "hatespeech"], df.at[i, "csType"]
    )


 Generating counterspeech using Flan-T5 on GPU...


Processing Hate Speech with Flan-T5:   0%|          | 0/2971 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Processing Hate Speech with Flan-T5: 100%|██████████| 2971/2971 [27:37<00:00,  1.79it/s]


In [9]:
df.to_csv("flan_t5_counterspeech.csv", index=False)

In [2]:
%pip install together 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 45.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.4 requires matplotlib>=3.8.0, but you have matplotlib 3.7.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [7]:
from together import Together


client = Together(api_key="1e89efbd5c30c3251be93683f4130b5c35b0146befe5f68bd006ae30f28430cf")  


def generate_counterspeech_togetherai(hate_speech, intent):
    """Generates counterspeech using LLaMA 3.3 70B via TogetherAI API"""
    
    
    prompt = f"""### Counterspeech Generation Task

You are an expert in counterspeech. Generate a **{intent}** counterspeech response to the following hate speech:

**Hate Speech Instance:**
{hate_speech}

---

### Expected Response
Your response should be a **{intent}** style counterspeech that directly addresses the hate speech instance.
"""

    try:
        
        response = client.chat.completions.create(
            model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",  
            messages=[{"role": "user", "content": prompt}],
            max_tokens=250,  
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            repetition_penalty=1,
            stop=["\n\n","<|eot_id|>", "<|eom_id|>"],
            stream=False  
        )

        
        return response.choices[0].message.content if response.choices else "No response generated."
    
    except Exception as e:
        print("Error with TogetherAI API:", e)
        return ""

df["Llama3_counterspeech"] = ""

print("\nGenerating counterspeech using TogetherAI LLaMA 3.3 70B...")


for i in tqdm(range(len(df)), desc="Processing Hate Speech"):
    df.at[i, "Llama3_counterspeech"] = generate_counterspeech_togetherai(
        df.at[i, "hatespeech"], df.at[i, "csType"]
    )







Generating counterspeech using TogetherAI LLaMA 3.3 70B...


Processing Hate Speech: 100%|██████████| 2971/2971 [6:52:18<00:00,  8.33s/it]    


In [10]:
df.to_csv("Llamazeroshot_counterspeech.csv", index=False)

In [6]:
newdf = df[["Llama3_counterspeech"]]
newdf.to_csv("Llama_cs.csv", index = False)

In [11]:
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)





Using device: cpu


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [16]:
FEW_SHOT_EXAMPLES = [
    {
        "hate_speech": "Maybe the UN could talk to those asian and african nations responsible for 90+ of the pollution in the oceans instead of insisting on this bullshit about climate change.",
        "intent": "Positive",
        "counterspeech": "You're right, ocean pollution is one of the main environmental problems we face, however, climate change is a global problem, and everyone should help to combat it, no matter where in the world you are from!"
    },
    {
        "hate_speech": "Maybe the UN could talk to those asian and african nations responsible for 90+ of the pollution in the oceans instead of insisting on this bullshit about climate change.",
        "intent": "Denouncing",
        "counterspeech": "The world would be a better place if people were more tolerant to each other."
    },
    {
        "hate_speech": "Maybe the UN could talk to those asian and african nations responsible for 90+ of the pollution in the oceans instead of insisting on this bullshit about climate change.",
        "intent": "Questioning",
        "counterspeech": "Doesn't everyone on the planet have a responsibility towards the climate?"
    },
    {
        "hate_speech": "Maybe the UN could talk to those asian and african nations responsible for 90+ of the pollution in the oceans instead of insisting on this bullshit about climate change.",
        "intent": "Informative",
        "counterspeech": "The us is the second most polluting country in the world - as the worldâ€™s biggest industrial and commercial power. we are all to blame here and need to work hand in hand to create sustainable change, as opposed to pointing the finger to others."
    },
    {
        "hate_speech": "Jews cannot be patriots, since their allegiance will always be to the state of Israel.",
        "intent": "Informative",
        "counterspeech": "You can have parents and grandparents born elsewhere, and still be a patriot for the country you were born in."
    },
    {
        "hate_speech": "Jews cannot be patriots, since their allegiance will always be to the state of Israel.",
        "intent": "Questioning",
        "counterspeech": "If you personally had some distant spanish or jewish heritage and you identified more as american surely that doesn't mean that you can't be an american patriot?"
    },
    {
        "hate_speech": "Jews cannot be patriots, since their allegiance will always be to the state of Israel.",
        "intent": "Denouncing",
        "counterspeech": "My best friend is jewish, she volunteers to help woman, disadvantages children, and her whole family work to make the uk a better place."
    },
    {
        "hate_speech": "Jews cannot be patriots, since their allegiance will always be to the state of Israel.",
        "intent": "Positive",
        "counterspeech": "It is possible to love one's country and one's religious home. the two are not exclusive."
    },
    {
        "hate_speech": "The British people should use those houses, not immigrants.",
        "intent": "Positive",
        "counterspeech": "This country is big enough for all of us. we should look forward to welcoming people into our countries if they want to. integration is not hard. give these people a break."
    },
    {
        "hate_speech": "The British people should use those houses, not immigrants.",
        "intent": "Questioning",
        "counterspeech": "Who are you to command what houses should be occupied by whom? who gave you this authority?"
    },
    {
        "hate_speech": "The British people should use those houses, not immigrants.",
        "intent": "Informative",
        "counterspeech": "There are enough empty homes in the uk for both migrants and veterans. migrants deserve to be treated humanely, try having some empathy. imagine if you were forced by circumstance to emigrate and have to live in poor conditions."
    },
    {
        "hate_speech": "The British people should use those houses, not immigrants.",
        "intent": "Denouncing",
        "counterspeech": "It is because of people like you that migrants do not feel safe in a place that they should call home. shame on you."
    }
    
    
]

def generate_counterspeech_flan_t5(hate_speech, intent):

    few_shot_examples = ""
    for example in FEW_SHOT_EXAMPLES:
        few_shot_examples += f"""
Hate Speech: {example['hate_speech']}
Intent: {example['intent']}
Expected Counterspeech ({example['intent']} intent): {example['counterspeech']}

---
"""

    prompt = f"""### Few-Shot Counterspeech Generation Task

Below are some examples of hate speech and their corresponding counterspeech responses based on different intents.

{few_shot_examples}

Now generate a counterspeech response for the following:

Hate Speech: {hate_speech}
Intent: {intent}
Expected Counterspeech ({intent} intent):"""

    # Tokenize and generate response
    inputs = tokenizer(prompt, return_tensors="pt").to(device)  
    with torch.no_grad():  
        outputs = model.generate(
            **inputs, 
            max_new_tokens=250,  
            repetition_penalty=1.2, 
            no_repeat_ngram_size=3,  
            temperature=0.5,   # Lower temperature for better intent control
            top_k=50,  
            top_p=0.9
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


df["FlanT5_few_shot_counterspeech"] = ""

print("\n Generating counterspeech using Flan-T5 (Few-Shot) on GPU...")

for i in tqdm(range(len(df)), desc="Processing Hate Speech with Flan-T5 Few-Shot"):
    df.at[i, "FlanT5_few_shot_counterspeech"] = generate_counterspeech_flan_t5_few_shot(
        df.at[i, "hatespeech"], df.at[i, "csType"]
    )


df.to_csv("flan_t5_few_shot_counterspeech.csv", index=False)


 Generating counterspeech using Flan-T5 (Few-Shot) on GPU...


Processing Hate Speech with Flan-T5 Few-Shot: 100%|██████████| 2971/2971 [46:31<00:00,  1.06it/s]
